In [1]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse

In [2]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [4]:
# iv: image option
length = 512
channel = 96
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [5]:
torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
eeg_dataset, splits_path = os.listdir(torch_models_dir)

In [6]:
eeg_dataset = os.path.join(torch_models_dir, eeg_dataset)
splits_path = os.path.join(torch_models_dir, splits_path)
print(eeg_dataset,'\n', splits_path)

D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models\imagenet40-1000-1.pth 
 D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models\imagenet40-1000-1_splits.pth


In [7]:
opt = {
    # Dataset options
    "iv": "image",
    "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "run": "none",
    "eeg_dataset": eeg_dataset,
    "splits_path": splits_path,
    "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"incremental",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.iv)

image


In [31]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
from data_loader import EEGDataset, Splitter, SplitterWithData
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_trainer import net_trainer
from p_values import *
from torchinfo import summary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def load_dataset(iv,
             offset,
             fold,
             eeg_dataset,
             splits_path,
             total,
             classes,
             classifier,
             batch_size,
             GPUindex,
             length, # 512
             channel, # 96
             min_CNN,
             opt,
             kind):
    for split_num in range(fold):
        
        # Load dataset
        dataset = EEGDataset(iv, eeg_dataset, classifier, map_idx = None)
        print("DONE: LOAD DATASET")
        # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
        if kind=="from-scratch":
            relabel = True
        if kind=="incremental":
            relabel = False
        if kind=="no-model-file":
            relabel = True
        loaders = {split: DataLoader(
            SplitterWithData(iv,
                        dataset,
                        splits_path,
                        classes,
                        split_num,
                        split,
                        relabel),
            batch_size = batch_size,
            drop_last = False,
            shuffle = True)
                for split in ["train", "val", "test"]}
        channel_idx = None    
        print("DONE: Create loaders for model")            
        return dataset, loaders

In [25]:
def Classifier(iv,
             offset,
             fold,
             eeg_dataset,
             splits_path,
             total,
             classes,
             classifier,
             batch_size,
             GPUindex,
             length, # 512
             channel, # 96
             min_CNN,
             opt,
             kind):
    if classifier=="LSTM":
        if kind=="from-scratch":
            output_size = 128
        if kind=="incremental":
            output_size = 128
        if kind=="no-model-file":
            output_size = 128
        net = classifier_LSTM(
            True,
            input_size = channel,
            lstm_layers = 1,
            lstm_size = 128,
            output1_size = 128,
            output2_size = None,
            GPUindex = GPUindex)
    elif classifier=="LSTM1":
        if kind=="from-scratch":
            output_size = 128
        if kind=="incremental":
            output_size = 128
        if kind=="no-model-file":
            output_size = 128
        net = classifier_LSTM(
            False,
            input_size = channel,
            lstm_layers = 1,
            lstm_size = 128,
            output1_size = 128,
            output2_size = None,
            GPUindex = GPUindex)
    elif classifier=="LSTM2":
        if kind=="from-scratch":
            output_size = len(classes)
        if kind=="incremental":
            output_size = total
        if kind=="no-model-file":
            output_size = len(classes)
        net = classifier_LSTM(
            False,
            input_size = channel,
            lstm_layers = 1,
            lstm_size = 128,
            output1_size = output_size,
            output2_size = None,
            GPUindex = GPUindex)
    elif classifier=="LSTM3":
        if kind=="from-scratch":
            output_size = len(classes)
        if kind=="incremental":
            output_size = total
        if kind=="no-model-file":
            output_size = len(classes)
        net = classifier_LSTM(
            True,
            input_size = channel,
            lstm_layers = 1,
            lstm_size = 128,
            output1_size = output_size,
            output2_size = None,
            GPUindex = GPUindex)
    elif classifier=="LSTM4":
        if kind=="from-scratch":
            output_size = len(classes)
        if kind=="incremental":
            output_size = total
        if kind=="no-model-file":
            output_size = len(classes)
        net = classifier_LSTM(
            True,
            input_size = channel,
            lstm_layers = 1,
            lstm_size = 128,
            output1_size = 128,
            output2_size = output_size,
            GPUindex = GPUindex)        
    elif classifier=="CNN":
        if length<min_CNN:
            return
        if kind=="from-scratch":
            output_size = len(classes)
        if kind=="incremental":
            output_size = total
        if kind=="no-model-file":
            output_size = len(classes)
        net = classifier_CNN(
            in_channel = channel,
            num_points = length,
                output_size = output_size)
    elif classifier=="EEGNet":
        if length<min_CNN:
            return
        if kind=="from-scratch":
            output_size = len(classes)
        if kind=="incremental":
            output_size = total
        if kind=="no-model-file":
            output_size = len(classes)
        net = classifier_EEGNet(channel, length)
    elif classifier=="SyncNet":
        if length<min_CNN:
            return 
        if kind=="from-scratch":
            output_size = len(classes)
        if kind=="incremental":
            output_size = total
        if kind=="no-model-file":
            output_size = len(classes)
        net = classifier_SyncNet(channel, length)
    elif classifier=="EEGChannelNet":
        if length<min_CNN:
            return
        if kind=="from-scratch":
            output_size = len(classes)
        if kind=="incremental":
            output_size = total
        if kind=="no-model-file":
            output_size = len(classes)
        net = classifier_EEGChannelNet(channel, length)
    print("DONE: CREATE TORCH CLASSIFIER")
    print(net)
    nonclasses = [i for i in range(output_size) if i not in classes]
    return net, nonclasses

In [26]:
# LSTM network
opt.classifier = "LSTM4"
opt.batch_size = 16
opt.kind = "from-scratch"
opt.run = "imagenet40-1000"
opt.fold = 1

In [12]:
dataset, loaders = load_dataset(opt.iv,
                             opt.offset,
                             opt.fold,
                             opt.eeg_dataset,
                             opt.splits_path,
                             n_classes,
                             classes,
                             opt.classifier,
                             opt.batch_size,
                             opt.GPUindex,
                             length,
                             channel,
                             min_CNN,
                             opt,
                             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [43]:
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
print(dataset.classes)

<class 'data_loader.EEGDataset'>
<class 'dict'>
3 [2000, 250, 250]


AttributeError: 'EEGDataset' object has no attribute 'classes'

In [33]:
net, nonclasses = Classifier(opt.iv,
                 opt.offset,
                 opt.fold,
                 opt.eeg_dataset,
                 opt.splits_path,
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.batch_size,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt,
                 opt.kind)
print(len(nonclasses))
summary(net, input_size=(1, 96, 512))

DONE: CREATE TORCH CLASSIFIER
classifier_LSTM(
  (lstm): LSTM(96, 128, batch_first=True)
  (output1): Linear(in_features=128, out_features=128, bias=True)
  (relu): ReLU()
  (output2): Linear(in_features=128, out_features=40, bias=True)
)
0


Layer (type:depth-idx)                   Output Shape              Param #
classifier_LSTM                          [1, 40]                   --
├─LSTM: 1-1                              [1, 512, 128]             115,712
├─Linear: 1-2                            [1, 128]                  16,512
├─ReLU: 1-3                              [1, 128]                  --
├─Linear: 1-4                            [1, 40]                   5,160
Total params: 137,384
Trainable params: 137,384
Non-trainable params: 0
Total mult-adds (M): 59.27
Input size (MB): 0.20
Forward/backward pass size (MB): 0.53
Params size (MB): 0.55
Estimated Total Size (MB): 1.27

In [34]:
model_path = (opt.iv+
                  "-"+
                  opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel)+
                  "-"+
                  str(0))
channel_idx=None
print(model_path)

image-LSTM4-512-96-0


In [35]:
if opt.kind=="from-scratch":
    accuracy_val, accuracy_test, counts_val, counts_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

epoch 1
Batch 1000: Loss=0.0; accuracy=0.9980000257492065
Batch 2000: Loss=0.0; accuracy=0.999000072479248
epoch 2
Batch 1000: Loss=0.0; accuracy=1.0


KeyboardInterrupt: 

In [ ]:
val =accuracy_val
test = accuracy_test
samples = counts_val+counts_test
accuracy = (val+test)/(1*2)
print("Validation accuracy: ", val)
print("Test accuracy: ", test)
print("Accuracy: ", accuracy) #????

In [ ]:
print(accuracy_val)